In [1]:
import requests
from requests import delete

import numpy as np
import re

import pickle
import uuid

import os

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from PIL import Image
import numpy as np
import pandas as pd

import io

In [2]:

from tksheet import Sheet

import tkinter as tk
from tkinter import filedialog
from re import sub
from tkinter import Y


from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

from  matplotlib.colors import LinearSegmentedColormap


In [3]:
from matplotlib.patches import Wedge, Circle, Rectangle
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.colors import ListedColormap
import colorcet as cc
from matplotlib.patches import Patch
from cmath import rect
import symbol


C:\Users\test\AppData\Local\Temp\ipykernel_25148\865205281.py:8: DeprecationWarning: The symbol module is deprecated and will be removed in future versions of Python
  import symbol


In [4]:
class pathway_viewer():
    def __init__(self,mother, image_array, rectangles, df_use, gene_dict, rect_df, title):
        self.mother = mother
        self.title = title
        self.cmap_custom=LinearSegmentedColormap.from_list('rg',["limegreen", "w", "r"], N=128) 
        self.image_array = image_array
        self.rectangles = rectangles
        self.rect_df = rect_df
        self.df_use = df_use
        self.gene_dict = gene_dict
        self.selected_id = ""
        self.selected_df = None
        self.selected_columns = []
        self.sub_view = None
        self.selected_col = -1
        self.view_scatter = 2
        
        self.x_min = 0
        self.y_min = 0
        self.x_max = self.image_array.shape[0]
        self.y_max = self.image_array.shape[1]
        self.x_min_now = 0
        self.y_min_now = 0
        self.x_max_now = self.image_array.shape[0]
        self.y_max_now = self.image_array.shape[1]
        self.open_viewer()
    def open_viewer(self):
        self.window_viewer = tk.Tk()
        self.window_viewer.title(self.title)
        self.window_viewer.geometry("1000x800")
        
        self.window_viewer_frame_1 = tk.Frame(self.window_viewer, width=800, height=800, bg="white")
        self.window_viewer_frame_2 = tk.Frame(self.window_viewer, width=200, height=800, bg="lightgray")
        self.window_viewer_frame_1.pack(side=tk.LEFT, fill=tk.BOTH, expand=False)
        self.window_viewer_frame_2.pack(side=tk.RIGHT, fill=tk.BOTH, expand=False)
        
        self.fig = Figure(figsize=(self.image_array.shape[1]/100, self.image_array.shape[0]/100), dpi=100)
        self.ax = self.fig.add_subplot(111)
        self.canvas_main = FigureCanvasTkAgg(self.fig, master=self.window_viewer_frame_1)
        self.canvas_main.get_tk_widget().pack()
        
        self.viewer_listbox = tk.Listbox(self.window_viewer_frame_2, selectmode = tk.MULTIPLE, height=300, exportselection = False)
        
        t = self.mother.table_dict[self.mother.table_name]
        signature_gene_df = t.df.set_index(t.index_col)[t.signature_gene_col].dropna()
        cluster_list = list(map(int,np.sort(signature_gene_df.unique())))
        self.selected_clusters_signature_gene = cluster_list[:]
        for col in cluster_list:
            self.viewer_listbox.insert(tk.END, col)
        self.viewer_listbox.bind("<<ListboxSelect>>", self.select_signature_gene_clusters)
        self.viewer_listbox.pack()
        self.view_pathway()
        self.canvas_main.draw()
        self.canvas_main.mpl_connect('button_press_event', self.on_canvas_click)
        self.canvas_main.mpl_connect('scroll_event', self.zoom_in_out)
        
        self.window_viewer.mainloop()
    def select_signature_gene_clusters(self,event=False):
        self.selected_clusters_signature_gene = [int(self.viewer_listbox.get(i)) for i in self.viewer_listbox.curselection()]
        self.view_pathway()
        
    def view_pathway(self):
        self.ax.cla()
        self.ax.imshow(self.image_array,origin='lower')
        self.ax.axis('off')
        self.ax.set_ylim(self.x_max_now, self.x_min_now)
        self.ax.set_xlim(self.y_min_now, self.y_max_now)
        
        symbol_list = self.df_use.index
        t = self.mother.table_dict[self.mother.table_name]
        signature_gene_df = t.df.set_index(t.index_col)[t.signature_gene_col].dropna()
        cmap_signature_gene =LinearSegmentedColormap.from_list('cmap_custom',cc.glasbey_category10[:len(signature_gene_df.unique())])
        cmap_signature_gene = [cmap_signature_gene(i) for i in np.linspace(0, 1, len(signature_gene_df.unique()))]

        for rect, key_id in zip(self.rectangles, self.gene_dict.keys()):
            target_entrez_ids = self.gene_dict[key_id]['hsa']
            x, y, width, height = rect
            # Rectangle 객체 생성 및 추가
            rect_patch = Rectangle(
                (x, y), width-x, height-y, 
                edgecolor='black', facecolor='none', linewidth=2
            )
            self.ax.add_patch(rect_patch)
            signature_cluster_all = []
            for i in target_entrez_ids:
                if int(i) in signature_gene_df.index:
                    if int(signature_gene_df.loc[int(i)]) in self.selected_clusters_signature_gene:
                        signature_cluster_all += [signature_gene_df.loc[int(i)]]
            if len(signature_cluster_all) == 1:
                rect_patch = Rectangle(
                    (x, y), width-x, height-y, 
                    edgecolor=cmap_signature_gene[int(signature_cluster_all[0])-1],
                    facecolor=cmap_signature_gene[int(signature_cluster_all[0])-1], linewidth=1, alpha = 0.5
                )
                self.ax.add_patch(rect_patch)
            elif len(signature_cluster_all) > 1:
                x_d = (width-x)/len(signature_cluster_all)
                for i in range(len(signature_cluster_all)):
                    rect_patch = Rectangle(
                        (x+x_d*i, y), x_d, height-y, 
                        edgecolor=cmap_signature_gene[int(signature_cluster_all[i])-1], 
                        facecolor=cmap_signature_gene[int(signature_cluster_all[i])-1], linewidth=1, alpha = 0.5
                    )
                    self.ax.add_patch(rect_patch)    
                    # on work to show signature genes
            
                
        
        for rect, key_id in zip(self.rectangles, self.gene_dict.keys()):
            target_entrez_ids = self.gene_dict[key_id]['hsa']
            x, y, width, height = rect
            if self.view_scatter == 0:
                value_all = []
                for i in target_entrez_ids:
                    if int(i) in symbol_list:
                        if self.df_use.notna().loc[int(i),self.selected_col]:
                            value_all += [self.df_use.loc[int(i),self.selected_col]]
                if len(value_all)>0:
                    self.ax.scatter([rect[0]],[rect[1]],s = 210,
                        cmap = self.cmap_custom, c = np.min(value_all), vmin=-1.2, vmax=1.2, linewidths=1, edgecolors='black')
                    self.ax.scatter([rect[0]],[rect[1]],s = 60,
                        cmap = self.cmap_custom, c = np.max(value_all), vmin=-1.2, vmax=1.2)
            elif self.view_scatter == 1:
                
                if len(self.selected_columns) > 0:
                    cmap_cluster=LinearSegmentedColormap.from_list('cmap_custom',cc.glasbey_category10[0:0+len(self.selected_columns)])
                    cmap_cluster = [cmap_cluster(i) for i in np.linspace(0, 1, len(self.selected_columns))]
                    norm = Normalize(vmin=-1.2, vmax=1.2)
                    sm = ScalarMappable(cmap=self.cmap_custom, norm=norm)
                    theta_r = 360/len(self.selected_columns)
                    for n,col in enumerate(self.selected_columns):
                        value_all = []
                        for i in target_entrez_ids:
                            if int(i) in symbol_list:
                                value_all += [self.df_use.loc[int(i),col]]
                        if len(value_all)>0:
                            color = cmap_cluster[n]
                            wedge1 = Wedge(
                                center=(rect[0], rect[1]),      # 중심 좌표
                                r=24,                # 반지름
                                theta1=theta_r * n,           # 시작 각도 (0도)
                                theta2=theta_r * (n+1),         # 종료 각도 (120도)
                                facecolor=color,
                            )
                            
                            color = sm.to_rgba(np.min(value_all))
                            wedge2 = Wedge(
                                center=(rect[0], rect[1]),      # 중심 좌표
                                r=20,                # 반지름
                                theta1=theta_r * n,           # 시작 각도 (0도)
                                theta2=theta_r * (n+1),         # 종료 각도 (120도)
                                facecolor=color,
                                edgecolor="black"
                            )
                            color = sm.to_rgba(np.max(value_all))
                            wedge3 = Wedge(
                                center=(rect[0], rect[1]),      # 중심 좌표
                                r=12,                # 반지름
                                theta1=theta_r * n,           # 시작 각도 (0도)
                                theta2=theta_r * (n+1),         # 종료 각도 (120도)
                                facecolor=color,
                                edgecolor="black"
                            )
                            self.ax.add_patch(wedge1)
                            self.ax.add_patch(wedge2)
                            self.ax.add_patch(wedge3)
        legend_items = [
            Patch(facecolor=i, label=f'C{n+1} A') for n,i in enumerate(cmap_signature_gene)
            ]
        self.ax.legend(handles=legend_items, loc='upper right', title='Signature gene')
        self.canvas_main.draw()
    def on_canvas_click(self,event):
        x, y = int(event.xdata), int(event.ydata)
        print(x,y)
        self.selected_ids = self.rect_df[(self.rect_df['x1']<=x)*(self.rect_df['x2']>=x)*(self.rect_df['y1']<=y)*(self.rect_df['y2']>=y)].index
        print(self.selected_ids)
        if len(self.selected_ids) == 0:
            self.selected_ids = self.rect_df.index
        if event.key == 'shift' and event.button == 1:
            for id_sub in self.selected_ids:
                self.selected_idxs_map += [int(i) for i in self.gene_dict[id_sub]['hsa'] if int(i) in self.df_use.index]
            self.selected_title += [self.df_use.loc[i,'Symbol'].tolist() for i in self.selected_idxs_map]
            
            print(self.selected_title)
            if self.sub_view is None:
                self.sub_viewer()
            try:
                self.listbox_gene.selection_clear(0,tk.END)
            except:
                self.sub_viewer()
            for i in self.selected_title:
                self.listbox_gene.select_set(self.listbox_gene.get(0,tk.END).index(i))
            self.select_df("")
        else:
            self.selected_idxs_map = []
            for id_sub in self.selected_ids:
                self.selected_idxs_map += [int(i) for i in self.gene_dict[id_sub]['hsa'] if int(i) in self.df_use.index]
            self.selected_title = self.df_use.loc[self.selected_idxs_map,'Symbol'].tolist()
            if self.sub_view is None:
                self.sub_viewer()
            try:
                self.listbox_gene.selection_clear(0,tk.END)
            except:
                self.sub_viewer()
            for i in self.selected_title:
                self.listbox_gene.select_set(self.listbox_gene.get(0,tk.END).index(i))
            self.select_df("")
        print(self.selected_title)
        
            
    def select_df(self,event):
        self.selected_columns = [self.listbox.get(i) for i in self.listbox.curselection()]
        self.selected_genes = [self.listbox_gene.get(i) for i in self.listbox_gene.curselection()]
        self.selected_genes_ids = [self.df_use.reset_index().set_index('Symbol').loc[i,'Unnamed: 0'] for i in self.selected_genes]
        self.selected_idxs = self.df_use.reset_index().set_index('Symbol').loc[self.selected_genes,self.mother.table_dict[self.mother.table_name].index_col]
        print(self.selected_genes_ids, self.selected_genes)
        self.sub_view_ax.cla()
        if (len(self.selected_genes) > 0) & (len(self.selected_columns) > 0):
            self.selected_df = self.df_use.loc[self.selected_idxs,self.selected_columns]
            self.sub_view_ax.imshow(self.selected_df,
                    interpolation='none', aspect = 'auto',
                    cmap=self.cmap_custom, vmin=-1, vmax=1)
            
            
            t = self.mother.table_dict[self.mother.table_name]
            signature_gene_df = t.df.set_index(t.index_col)[t.signature_gene_col].dropna()
            
            cmap_signature_gene =LinearSegmentedColormap.from_list('cmap_custom',cc.glasbey_category10[:len(signature_gene_df.unique())])
            cmap_signature_gene = [cmap_signature_gene(i) for i in np.linspace(0, 1, len(signature_gene_df.unique()))]
            signature_gene_df = signature_gene_df.loc[[int(i) for i in self.selected_idxs if int(i) in signature_gene_df.index]]
            
            for i in range(len(self.selected_df)):
                if self.selected_df.index[i] in signature_gene_df.index:
                    rect = Rectangle((-1,i-0.5),0.5,1,
                                     facecolor=cmap_signature_gene[int(signature_gene_df.loc[self.selected_df.index[i]])-1],
                                     edgecolor='none')
                    self.sub_view_ax.add_patch(rect)    
            if len(self.selected_df)<20:
                self.sub_view_ax.set_yticks(range(len(self.selected_df)), [f"{i}\n{j}" for i,j in zip(self.selected_genes_ids,self.selected_genes)])
                for i in range(len(self.selected_columns)):
                    for j in range(len(self.selected_genes)):
                        self.sub_view_ax.text(i, j, round(self.selected_df.iloc[j,i],3), ha='center', va='center')
            self.sub_view_ax.set_xticks(range(len(self.selected_columns)), self.selected_columns)
            self.sub_view_ax.tick_params(axis='x', rotation=45)
            self.sub_view_ax.set_xlim(-1, len(self.selected_columns)-0.5)
            self.sub_view_ax.set_ylim(-0.5, len(self.selected_df)-0.5)
            self.sub_view_canvas.draw()
            
            # now boxplot
            self.sub_view_ax_boxplot.cla()
            A = [self.selected_df[col_i].dropna() for col_i in self.selected_df.columns]
            medianprops = dict(linestyle='-.', linewidth=2.5, color='white')
            bplot = self.sub_view_ax_boxplot.boxplot(A, patch_artist= True, sym = '.', medianprops=medianprops, widths = 0.7)
    
            for n,A_val in enumerate(A):
                self.sub_view_ax_boxplot.scatter([n+1+np.clip(np.random.randn(len(A_val))*0.1,-0.5,0.5)], A_val, s = 2, c = 'k', zorder = 3, alpha = 0.1)
            self.sub_view_canvas_boxplot.draw()
    def sub_viewer(self):
        self.sub_view = tk.Tk()
        self.sub_view.title("Selected gene expression")
        self.sub_view.geometry("800x800")
        
        self.sub_view_left_1 = tk.Frame(self.sub_view, width=200, height=400, bg="white")
        self.sub_view_left_2 = tk.Frame(self.sub_view, width=200, height=400, bg="white")
        self.sub_view_right = tk.Frame(self.sub_view, width=400, height=400, bg="lightgray")
        self.sub_view_left_1.pack(side=tk.LEFT, fill=tk.BOTH, expand=False)
        self.sub_view_left_2.pack(side=tk.LEFT, fill=tk.BOTH, expand=False)
        self.sub_view_right.pack(side=tk.RIGHT, fill=tk.BOTH, expand=False)
        self.sub_view_fig = Figure(figsize=(4, 4), dpi=100)
        self.sub_view_ax = self.sub_view_fig.add_subplot(111)
        self.sub_view_canvas = FigureCanvasTkAgg(self.sub_view_fig, master=self.sub_view_right)
        self.sub_view_canvas.mpl_connect("button_press_event",self.on_sub_canvas_click)
        self.sub_view_canvas.get_tk_widget().pack()
        self.sub_view_fig_boxplot = Figure(figsize=(4, 4), dpi=100)
        self.sub_view_ax_boxplot = self.sub_view_fig_boxplot.add_subplot(111)
        self.sub_view_canvas_boxplot = FigureCanvasTkAgg(self.sub_view_fig_boxplot, master=self.sub_view_right)
        self.sub_view_canvas_boxplot.get_tk_widget().pack()
        
        self.listbox = tk.Listbox(self.sub_view_left_1, selectmode = tk.MULTIPLE, height=0, exportselection = False)
        for col in self.df_use.columns:
            self.listbox.insert(tk.END, col)
        self.listbox.bind("<<ListboxSelect>>", self.select_df)
        self.listbox.pack()
        
        self.listbox_gene = tk.Listbox(self.sub_view_left_2, selectmode = tk.MULTIPLE, height=0, exportselection = False)
        for id_sub in self.rect_df.index:
            for j in  [self.df_use.loc[int(i),'Symbol'] for i in self.gene_dict[id_sub]['hsa'] if int(i) in self.df_use.index]:
                if j not in self.listbox_gene.get(0,tk.END):
                    self.listbox_gene.insert(tk.END, j)
        self.listbox_gene.bind("<<ListboxSelect>>", self.select_df)
        self.listbox_gene.pack()
        
        
        self.sub_view.mainloop()
        
    def zoom_in_out(self,event):
        x, y = int(event.xdata), int(event.ydata)
        if event.button == 'up': # zoom in
            self.x_max_now = int(y+(self.x_max_now-y)*8/10)
            self.x_min_now = int(y-(y-self.x_min_now)*8/10)
            if self.x_min_now >= self.x_max_now:
                self.x_max_now = self.x_min_now + 1 
            self.y_max_now = int(x+(self.y_max_now-x)*8/10)
            self.y_min_now = int(x-(x-self.y_min_now)*8/10)
            if self.y_min_now >= self.y_max_now:
                self.y_max_now = self.y_min_now + 1 
        elif event.button == 'down': # zoom out
            self.x_max_now = int(y+(self.x_max_now-y)*12/10)
            self.x_min_now = int(y-(y-self.x_min_now)*12/10)
            self.x_max_now = min(self.x_max_now,self.x_max)
            self.x_min_now = max(self.x_min_now,self.x_min)
            self.y_max_now = int(x+(self.y_max_now-x)*12/10)
            self.y_min_now = int(x-(x-self.y_min_now)*12/10)
            self.y_max_now = min(self.y_max_now,self.y_max)
            self.y_min_now = max(self.y_min_now,self.y_min)
        self.view_pathway()
        print(self.x_min_now, self.x_max_now, self.y_min_now, self.y_max_now)
    def on_sub_canvas_click(self,event):
        x, y = event.xdata, event.ydata
        
        selected_col = self.selected_columns[int((x+0.5)//1)]
        print(selected_col)
        if self.selected_col == selected_col:
            self.view_scatter += 1
            self.view_scatter %= 3
        else:
            self.selected_col = selected_col
            self.view_scatter = 0
        self.view_pathway()
        

In [5]:
class Table_loader():
    def __init__(self):
        self.table_dict = {}
        self.table_name = ""
    def load_table(self):
        self.window_table = tk.Tk()
        
        self.window_table_frame_1 = tk.Frame(self.window_table, width=500, height=200, bg="white")
        self.window_table_frame_2 = tk.Frame(self.window_table, width=500, height=400, bg="lightgray")
        self.window_table_frame_1.pack(side=tk.TOP, fill=tk.BOTH, expand=False)
        self.window_table_frame_2.pack(side=tk.BOTTOM, fill=tk.BOTH, expand=False)
        self.window_table.title("Main df")
        self.window_table.geometry("1200x600")
        
        self.window_table_menubar = tk.Menu(self.window_table)
        self.window_table_filemenu = tk.Menu(self.window_table_menubar, tearoff=0)
        self.window_table_filemenu.add_command(label="Open File", command=self.open_file)
        # self.window_table_filemenu.add_command(label="Save Map", command=self.save_map)
        self.window_table_menubar.add_cascade(label="File", menu=self.window_table_filemenu)
        self.window_table.config(menu=self.window_table_menubar)
        self.table_listbox = tk.Listbox(self.window_table_frame_1, selectmode = 'single', height=0, exportselection = False)
        if len(self.table_dict) > 0:
            for key in self.table_dict.keys():
                self.table_listbox.insert(tk.END, key)
        self.table_listbox.bind("<<ListboxSelect>>", self.show_table)
        self.table_listbox.pack()
        self.sheet = Sheet(
            self.window_table_frame_2,
            theme="light green",
            height=400,
            width=1000
        )
        self.sheet.set_sheet_data([["No data available"]])
        self.sheet.grid()
        self.sheet.enable_bindings(("single_select",
                       "column_select",
                       "row_select",
                       "column_width_resize",
                       "arrowkeys",
                       "right_click_popup_menu",
                       "rc_select",
                       "rc_insert_row",
                       "rc_delete_row",
                       "copy",
                       "cut",
                       "paste",
                       "delete",
                       "undo",
                       "edit_cell",
                       "drag_select"))
        self.sheet_popup_menu = tk.Menu(self.window_table_frame_2, tearoff=0)
        self.set_sheet_popup_menu()
        
        
        self.sheet.bind("<Button-3>", self.show_popup_menu)
        
        # Bind Right-Click Event
        self.window_table.mainloop()
    
    def set_sheet_popup_menu(self):
        self.sheet_popup_menu.delete(0, tk.END)
        if self.table_name != "":
            self.sheet_popup_menu.add_command(label=f"index: {self.table_dict[self.table_name].index_col}")
        else:
            self.sheet_popup_menu.add_command(label="index: None")
        self.sheet_popup_menu.add_command(label="set as index", command=self.set_index)
        self.sheet_popup_menu.add_command(label="set as signature gene", command=self.set_signature_gene)
        self.sheet_popup_menu.add_command(label="show all info", command=self.show_table_info)
        
    def set_signature_gene(self, event = False):
        self.table_dict[self.table_name].signature_gene_col = self.table_dict[self.table_name].df.columns[list(self.sheet.get_selected_columns())[0]]
        self.set_sheet_popup_menu()
        
    def show_table_info(self, event = False):
        print(f" table name: {self.table_dict[self.table_name].df_name}")
        print(f" table format: {self.table_dict[self.table_name].df_format}")
        print(f" table shape: {self.table_dict[self.table_name].df.shape}")
        print(f" index : {self.table_dict[self.table_name].index_col}")
        print(f" signature gene : {self.table_dict[self.table_name].signature_gene_col}")
        
    def show_popup_menu(self, event):
        self.sheet_popup_menu.post(event.x_root, event.y_root)
    
    def set_index(self, event = False):
        print(self.sheet.get_selected_columns())
        self.table_dict[self.table_name].index_col = self.table_dict[self.table_name].df.columns[list(self.sheet.get_selected_columns())[0]]
        self.set_sheet_popup_menu()
        self.set_customizer_df()
        
    def open_file(self, event = False):
        self.file_name = filedialog.askopenfilename()
        if self.file_name.endswith('.csv'):
            self.table_dict[self.file_name.split('/')[-1].replace('.csv','')] = table_unit(self.file_name,'csv')
        elif self.file_name.endswith('.xlsx'):
            self.table_dict[self.file_name.split('/')[-1].replace('.xlsx','')] = table_unit(self.file_name,'xlsx')
            
        self.show_table_list()
    
    
    def show_table_list(self):
        self.table_listbox.delete(0,tk.END)
        if len(self.table_dict) > 0:
            for key in self.table_dict.keys():
                self.table_listbox.insert(tk.END, key)
                
    def show_table(self, event = False):
        self.table_name = self.table_listbox.get(self.table_listbox.curselection())
        self.sheet.set_sheet_data(
            data=self.table_dict[self.table_name].df.values.tolist())
        self.sheet.set_header_data(self.table_dict[self.table_name].df.columns)
        self.sheet.set_index_data(self.table_dict[self.table_name].df.index)

class table_unit():
    def __init__(self, df_name, df_format):
        self.df_name = df_name
        self.df_format = df_format
        if self.df_format == 'csv':
            self.df = pd.read_csv(self.df_name)
        elif self.df_format == 'xlsx':
            self.df = pd.read_excel(self.df_name)
        self.index_col = None
        self.signature_gene_col = None

In [6]:
class KEGG_caller():
    def __init__(self):
        self.viewer = {}
        self.pathway_id = ""
        self.ptn_id = r'id="(.*?)"'
        self.ptn_coords = r'coords="(.*?)"'
        self.ptn_href = r'href="(.*?)"'
        self.ptn_title = r'title="(.*?)"'
        self.ptn_shape = r'shape="(.*?)"'
        self.on_searching = False


        pass 
    def request_KEGG_data(self,event = False):
        if self.on_searching:
            return
        self.on_searching = True
        self.pathway_id = self.pathway_input.get()
        if self.pathway_id not in self.viewer.keys():
            url = f'https://www.genome.jp/pathway/{self.pathway_id}'
            html = requests.get(url)
            if html.status_code != 200:
                print('No such pathway')
                self.on_searching = False
                return
            
            A = html.text.split('\n')
            gene_list = A[np.where([i.startswith('\t<map') for i in A])[0][0]+1:np.where([i.startswith('\t</map') for i in A])[0][0]]
            pathway_title = [i.replace('DEFINITION  ','') for i in A if i.startswith('DEFINITION')][0]
            response_img = requests.get(f'https://www.kegg.jp/kegg/pathway/hsa/{self.pathway_id}.png')

            self.gene_dict = {}
            for i in gene_list:
                coords = re.findall(self.ptn_coords, i)[0]
                if re.findall(self.ptn_shape, i)[0] != 'circle':
                    try:
                        self.gene_dict[re.findall(self.ptn_id, i)[0]] = {
                            'coords': list(map(int,coords.split(','))),
                            'hsa': [j.replace('+','') for j in re.findall(self.ptn_href, i)[0].split('hsa:')[1:]],
                            'title': re.findall(r'\((.*?)\)', re.findall(self.ptn_title, i)[0])
                        }
                    except:
                        pass
                    
            self.rectangles = [i['coords'] for i in self.gene_dict.values()]
            self.rectangles = [list(np.reshape(i,(-1,2)).min(axis=0))+list(np.reshape(i,(-1,2)).max(axis=0)) if len(i) > 4 else i for i in self.rectangles]
            self.rect_ids = list(self.gene_dict.keys())
            self.rect_df = pd.DataFrame(self.rectangles, columns = ['x1','y1','x2','y2'], index = self.rect_ids)
            
            image = Image.open(io.BytesIO(response_img.content))
            self.image_array = np.array(image)
            with open(f'./KEGG_folder/{self.pathway_id}.pkl','wb') as f:
                pickle.dump([self.image_array, self.rectangles, self.gene_dict, self.rect_df], f)
            self.saved_pathway_df.loc[self.pathway_id] = pathway_title
            self.saved_pathway_df.to_csv('./KEGG_folder/KEGG_ID.csv')
            self.on_searching = False
            self.viewer[self.pathway_id] = pathway_viewer(self, self.image_array, self.rectangles, self.table_dict[self.table_name].df.set_index(self.table_dict[self.table_name].index_col), self.gene_dict, self.rect_df, f"{self.pathway_id}: {pathway_title}")
        else:
            self.on_searching = False
            self.viewer[self.pathway_id].open_viewer()
            
    def get_pathway(self):
        self.pathway_mapper = tk.Tk()
        self.pathway_mapper.bind("<Return>", self.request_KEGG_data)
        self.pathway_mapper.title('Pathway Img')
        self.pathway_mapper.geometry("1000x300")
        self.pathway_input = tk.Entry(self.pathway_mapper, width = 100)
        self.pathway_input.bind("<Key>", self.filter_list)
        self.pathway_input.insert(0, self.pathway_id)
        self.pathway_input.icursor(0)
        self.pathway_input.pack()
        self.pathway_mapper_btn_1 = tk.Button(self.pathway_mapper, text = 'Enter', command = self.request_KEGG_data)
        self.pathway_mapper_btn_1.pack(pady=20)
        
        self.pathway_mapper_listbox = tk.Listbox(self.pathway_mapper, selectmode = 'single', height=0, exportselection = False, width = 100)
        self.saved_pathway_df = pd.read_csv('./KEGG_folder/KEGG_ID.csv', index_col = 0)
        for i in os.listdir('./KEGG_folder'):
            if i.endswith('.pkl'):
                self.pathway_mapper_listbox.insert(tk.END, f"{i.replace('.pkl','')}: {self.saved_pathway_df.loc[i.replace('.pkl',''),'Pathway']}")
                
        self.pathway_mapper_listbox.bind("<<ListboxSelect>>", self.select_saved_pathway)
        self.pathway_mapper_listbox.pack()
        self.pathway_mapper.mainloop()

    def filter_list(self,event=False):
        self.pathway_mapper_listbox.delete(0,tk.END)
        filter_text = self.pathway_input.get()
        if (event.char) == '\x08':
            filter_text = filter_text[:-1]
        else:
            filter_text = filter_text + event.char
        for i in os.listdir('./KEGG_folder'):
            if i.endswith('.pkl'):
                if self.saved_pathway_df.loc[i.replace('.pkl',''),'Pathway'].lower().find(filter_text.lower()) != -1:
                    self.pathway_mapper_listbox.insert(tk.END, f"{i.replace('.pkl','')}: {self.saved_pathway_df.loc[i.replace('.pkl',''),'Pathway']}")
          
    
    def select_saved_pathway(self,event = False):
        pathway_title = self.pathway_mapper_listbox.get(self.pathway_mapper_listbox.curselection())
        self.pathway_id = pathway_title.split(':')[0]
        self.image_array, self.rectangles, self.gene_dict, self.rect_df = pickle.load(open(f'./KEGG_folder/{self.pathway_id}.pkl','rb'))
        self.viewer[self.pathway_id] = pathway_viewer(self, self.image_array, self.rectangles, self.table_dict[self.table_name].df.set_index(self.table_dict[self.table_name].index_col), self.gene_dict, self.rect_df, pathway_title)

In [7]:
class Customized_mapper():
    def __init__(self):
        self.mapper_dict = {"Untitled_map":{'file_name':'Untitled_map',
                                            'components':{},
                                            'relatives':pd.DataFrame(columns=['mom','child','child_label']),
                                            'x_lim':[0,100],
                                            'y_lim':[0,100]}}
        self.selected_map = "Untitled_map"
        self.selected_map_data = {'file_name':'Untitled_map',
                                  'components':{},
                                  'relatives':pd.DataFrame(columns=['mom','child','child_label'])}
        self.selected_map_data_df = pd.DataFrame(columns= ['x','y','width','height'])
        self.mode_now = "view" # new_comp, view
        self.onclick_comp = False
        self.ondrag_comp = False
        
        self.detail_types = {'label':str,
                             'x':float,'y':float,
                             'width':float,'height':float,
                             'color':str,'alpha':float,'fontsize':int
                             }
        
        self.on_mouse_click = False
        self.on_mouse_drag = False
        
        self.cmap_custom_customize=LinearSegmentedColormap.from_list('rg',["limegreen", "w", "r"], N=128)
        
        ## 컬러바 생성
    
    def set_customizer_df(self):
        self.df_use = self.table_dict[self.table_name].df.set_index(self.table_dict[self.table_name].index_col)
        
    
    def load_customizer(self):
        self.customize_mapper = tk.Tk()
        self.customize_mapper.title('Customizer')
        self.customize_mapper.geometry("500x500")
        self.customize_mapper.bind("<Control-n>", self.change_mode)
        
        
        self.customize_mapper_menubar = tk.Menu(self.customize_mapper)
        self.filemenu = tk.Menu(self.customize_mapper_menubar, tearoff=0)
        self.filemenu.add_command(label="Open Map", command=self.load_map)
        self.filemenu.add_command(label="Save Map", command=self.save_map)
        self.customize_mapper_menubar.add_cascade(label="File", menu=self.filemenu)
        self.customize_mapper.config(menu=self.customize_mapper_menubar)
        
        self.customize_mapper_listbox = tk.Listbox(self.customize_mapper, selectmode = 'single', height=0, exportselection = False)
        self.show_loaded_maps()

        self.customized_map_fig = Figure(figsize = (10,10))
        self.customized_map_ax = self.customized_map_fig.add_subplot(111)
        self.customize_mapper_canvas = FigureCanvasTkAgg(self.customized_map_fig, master=self.customize_mapper)
        self.customize_mapper_canvas.get_tk_widget().pack()
        self.customize_mapper_canvas.mpl_connect('button_press_event', self.on_custom_canvas_click)
        self.customize_mapper_canvas.mpl_connect('button_release_event', self.on_custom_canvas_release)
        self.customize_mapper_canvas.mpl_connect('motion_notify_event', self.on_custom_canvas_drag)
        self.customize_mapper_canvas.mpl_connect('scroll_event', self.zoom_custom_canvas)
        self.load_customizer_detail()
        self.load_cluster_selecting_widget()
        
    def load_cluster_selecting_widget(self):
        self.customize_cluster_selection = tk.Tk()
        self.customize_cluster_selection.title('Cluster_selection')
        self.customize_cluster_selection.geometry("300x500")
        
        self.listbox_customize = tk.Listbox(self.customize_cluster_selection, selectmode = tk.SINGLE, height=0, exportselection = False)
        for col in self.df_use.columns:
            self.listbox_customize.insert(tk.END, col)
        self.listbox_customize.bind("<<ListboxSelect>>", self.select_cluster_customize)
        self.listbox_customize.pack()
    
        # self.gene_dict
        # now
    def select_cluster_customize(self,event = False):
        self.selected_columns = [self.listbox_customize.get(i) for i in self.listbox_customize.curselection()]
        if len(self.selected_columns) > 0:
            for i in self.selected_map_data['components'].keys():
                if self.selected_map_data['components'][i]['label'] in self.df_use['Symbol'].values:
                    val = self.df_use.set_index('Symbol').loc[self.selected_map_data['components'][i]['label'],self.selected_columns[0]]
                    val = (np.clip(val,-1,1)+1)*64
                    print(val)
                    self.selected_map_data['components'][i]['color_cluster'] = self.cmap_custom_customize(int(val))
                else:
                    try:
                        self.selected_map_data['components'][i].pop('color_cluster') 
                    except:
                        pass
        else:
            for i in self.selected_map_data['components'].keys():
                try:
                    self.selected_map_data['components'][i].pop('color_cluster') 
                except:
                    pass
        self.view_customized_map()   
    def zoom_custom_canvas(self,event):
        x_gap = max((self.mapper_dict[self.selected_map]['x_lim'][1] - self.mapper_dict[self.selected_map]['x_lim'][0])/4, 5)
        x_r = (event.xdata-self.mapper_dict[self.selected_map]['x_lim'][0])/(self.mapper_dict[self.selected_map]['x_lim'][1] - self.mapper_dict[self.selected_map]['x_lim'][0])
        y_gap = max((self.mapper_dict[self.selected_map]['y_lim'][1] - self.mapper_dict[self.selected_map]['y_lim'][0])/4, 5)
        y_r = (event.ydata-self.mapper_dict[self.selected_map]['y_lim'][0])/(self.mapper_dict[self.selected_map]['y_lim'][1] - self.mapper_dict[self.selected_map]['y_lim'][0])
        if event.button == 'down':
            self.mapper_dict[self.selected_map]['x_lim'][0] = int(self.mapper_dict[self.selected_map]['x_lim'][0] - x_gap*x_r)
            self.mapper_dict[self.selected_map]['x_lim'][1] = int(self.mapper_dict[self.selected_map]['x_lim'][1] + x_gap*(1-x_r))
            self.mapper_dict[self.selected_map]['y_lim'][0] = int(self.mapper_dict[self.selected_map]['y_lim'][0] - y_gap*y_r)
            self.mapper_dict[self.selected_map]['y_lim'][1] = int(self.mapper_dict[self.selected_map]['y_lim'][1] + y_gap*(1-y_r))
        elif event.button == 'up':
            self.mapper_dict[self.selected_map]['x_lim'][0] = int(self.mapper_dict[self.selected_map]['x_lim'][0] + x_gap*x_r)
            self.mapper_dict[self.selected_map]['x_lim'][1] = int(self.mapper_dict[self.selected_map]['x_lim'][1] - x_gap*(1-x_r))
            self.mapper_dict[self.selected_map]['y_lim'][0] = int(self.mapper_dict[self.selected_map]['y_lim'][0] + y_gap*y_r)
            self.mapper_dict[self.selected_map]['y_lim'][1] = int(self.mapper_dict[self.selected_map]['y_lim'][1] - y_gap*(1-y_r))
            d = 10
            self.mapper_dict[self.selected_map]['x_lim'][0] = min(self.mapper_dict[self.selected_map]['x_lim'][0],self.mapper_dict[self.selected_map]['x_lim'][1]-d)
            self.mapper_dict[self.selected_map]['y_lim'][0] = min(self.mapper_dict[self.selected_map]['y_lim'][0],self.mapper_dict[self.selected_map]['y_lim'][1]-d)
        self.view_customized_map()
            
    def load_customizer_detail(self):
        self.customize_detail = tk.Tk()
        self.customize_detail.title('Details')
        self.customize_detail.geometry("300x500")
        
        self.customize_detail.labels = {}
        self.customize_detail.entries = {}
        
        for i in ['label','x','y','width','height' ,'color','alpha','fontsize']:
            self.customize_detail.labels[i] = tk.Label(self.customize_detail, text = i)
            self.customize_detail.labels[i].pack()
            self.customize_detail.entries[i] = tk.Entry(self.customize_detail, name = i)
            self.customize_detail.entries[i].pack()
        
        
        self.customize_detail.bind("<Return>", self.update_detail)
        
        self.customize_detail.btn_add_child = tk.Button(self.customize_detail, text = 'Add Child', command = self.add_child)
        self.customize_detail.btn_add_child.pack()
        self.customize_detail.listbox_child = tk.Listbox(self.customize_detail, selectmode = 'single', height=0, exportselection = False)
        self.customize_detail.listbox_child.bind("<<Delete>>", self.delete_child)
        self.customize_detail.listbox_child.pack()
        
    def update_detail(self,event):
        if self.onclick_comp:
            for i in ['x','y','width','height','color','alpha','label','fontsize']:
                self.mapper_dict[self.selected_map]['components'][self.onclick_comp][i] = self.detail_types[i](self.customize_detail.entries[i].get())
            self.view_customized_map()
        
    def load_detail(self,event = False):    
        if self.onclick_comp:
            for i in ['x','y','width','height','color','alpha','label','fontsize']:
                self.customize_detail.entries[i].delete(0,tk.END)
                self.customize_detail.entries[i].insert(0,self.selected_map_data['components'][self.onclick_comp][i])
            self.customize_detail.listbox_child.delete(0,tk.END)
            for i in self.mapper_dict[self.selected_map]['relatives'][(self.mapper_dict[self.selected_map]['relatives']['mom'] == self.onclick_comp)]['child']:
                self.customize_detail.listbox_child.insert(tk.END, i)
        self.customize_detail.focus_set()
        
        
    def add_child(self,event = False):
        self.mode_now = "add_child"
        
    def delete_child(self,event = False):
        child_idx = self.customize_detail.listbox_child.curselection()
        child_name = self.customize_detail.listbox_child.get(child_idx)
        child_id = child_name.split(": ")[-1]
        self.customize_detail.listbox_child.delete(child_idx)
        self.mapper_dict[self.selected_map]['relatives'] = self.mapper_dict[self.selected_map]['relatives'][(self.mapper_dict[self.selected_map]['relatives']['mom'] != self.onclick_comp)|(self.mapper_dict[self.selected_map]['relatives']['child'] != child_id)]
        
    def show_loaded_maps(self):
        self.customize_mapper_listbox.delete(0,tk.END)
        for i in self.mapper_dict.keys():
            self.customize_mapper_listbox.insert(tk.END, i)
        self.customize_mapper_listbox.select_set(0)
        self.customize_mapper_listbox.bind("<<ListboxSelect>>", self.view_customized_map)
        self.customize_mapper_listbox.pack()
        
    def load_map(self):
        self.file_path = filedialog.askopenfilename()
        if self.file_path.endswith('.pkl'):
            file_data = pickle.load(open(self.file_path,'rb'))
            if file_data['file_name'] not in self.mapper_dict.keys():
                self.mapper_dict[file_data['file_name']] = file_data
                self.show_loaded_maps()
                self.customize_mapper_listbox.selection_clear(0,tk.END)
                self.customize_mapper_listbox.select_set(self.customize_mapper_listbox.get(0,tk.END).index(file_data['file_name']))
    def save_map(self):
        file_path = filedialog.asksaveasfilename(initialdir='./', defaultextension='.pkl')
        file_name = file_path.split('/')[-1].replace('.pkl','')
        self.mapper_dict[self.selected_map]['file_name'] = file_name
        self.mapper_dict[file_name] = self.mapper_dict.pop(self.selected_map)
        self.selected_map = file_name
        
        pickle.dump(self.mapper_dict[self.selected_map], open(file_path,'wb'))
        self.show_loaded_maps()
        self.view_customized_map()
        
    def view_customized_map(self,event = False):
        self.selected_map = self.customize_mapper_listbox.get(self.customize_mapper_listbox.curselection())
        self.selected_map_data = self.mapper_dict[self.selected_map]
                
        self.customized_map_ax.cla()
        
        
        if len(self.selected_map_data['components'])>0:
            for j, i in self.selected_map_data['components'].items():
                if 'color_cluster' in i.keys():
                    Rectangle_now = Rectangle((i['x'],i['y']), i['width'], i['height'], color = i['color_cluster'], alpha = i['alpha'])
                else:
                    Rectangle_now = Rectangle((i['x'],i['y']), i['width'], i['height'], color = i['color'], alpha = i['alpha'])
                if self.onclick_comp == j:
                    Rectangle_now.set_edgecolor('yellow')
                    Rectangle_now.set_linewidth(2)
                else:
                    Rectangle_now.set_edgecolor('black')
                    Rectangle_now.set_linewidth(0.5)
                self.customized_map_ax.add_patch(Rectangle_now)
                self.customized_map_ax.text(i['x']+i['width']/2,i['y']+i['height']/2,i['label'], fontsize = i['fontsize'], ha = 'center', va = 'center')
                self.selected_map_data_df.loc[j] = [i['x'],i['y'],i['width'],i['height']]
                    
        self.customized_map_ax.set_xlim(self.selected_map_data['x_lim'][0], self.selected_map_data['x_lim'][1])
        self.customized_map_ax.set_ylim(self.selected_map_data['y_lim'][0], self.selected_map_data['y_lim'][1])
        
        self.customize_mapper_canvas.draw()
            
    def change_mode(self,event = False):
        self.mode_now = [i for i in ["view","new_comp"] if i != self.mode_now][0]
        
    def on_custom_canvas_click(self, event = False):
        if self.on_mouse_click == False:
            self.on_mouse_click = True
            self.click_start_x = event.xdata
            self.click_start_y = event.ydata
            
    def on_custom_canvas_drag(self, event = False):
        
        if self.on_mouse_drag:
            if self.ondrag_comp == False:
                self.mapper_dict[self.selected_map]['x_lim'][0] -= int(event.xdata - self.click_start_x)
                self.mapper_dict[self.selected_map]['x_lim'][1] -= int(event.xdata - self.click_start_x)
                self.mapper_dict[self.selected_map]['y_lim'][0] -= int(event.ydata - self.click_start_y)
                self.mapper_dict[self.selected_map]['y_lim'][1] -= int(event.ydata - self.click_start_y)
                self.click_start_x = event.xdata
                self.click_start_y = event.ydata
                self.on_mouse_drag = False
            else:
                self.mapper_dict[self.selected_map]['components'][self.ondrag_comp]['x'] = event.xdata - self.mapper_dict[self.selected_map]['components'][self.ondrag_comp]['width']/2
                self.mapper_dict[self.selected_map]['components'][self.ondrag_comp]['y'] = event.ydata - self.mapper_dict[self.selected_map]['components'][self.ondrag_comp]['height']/2
                self.load_detail()
        else:
            if self.on_mouse_click:
                if max(np.abs(event.x- self.click_start_x), np.abs(event.y- self.click_start_y)) > 5:
                    self.on_mouse_drag = True
                if self.onclick_comp:
                    if (np.abs(self.click_start_x - self.selected_map_data_df.loc[self.onclick_comp,'x'] - self.selected_map_data_df.loc[self.onclick_comp,'width']/2) < self.selected_map_data_df.loc[self.onclick_comp,'width']/2
                        ) & (np.abs(self.click_start_y - self.selected_map_data_df.loc[self.onclick_comp,'y']-self.selected_map_data_df.loc[self.onclick_comp,'height']/2) < self.selected_map_data_df.loc[self.onclick_comp,'height']/2):
                        self.ondrag_comp = self.onclick_comp
                    else:
                        self.ondrag_comp = False
        if self.mode_now =='new_comp':
            self.mapper_dict[self.selected_map]['components']['temp'] = {'x':event.xdata-10, 'y':event.ydata-5,
                                                                        'width':20, 'height':10, 
                                                                        'color':'white', 'alpha':0.4,
                                                                        'label':'', 'fontsize' : 0}
        self.view_customized_map()
        
    def on_custom_canvas_release(self, event = False):
        if not self.on_mouse_drag:
            if self.mode_now == "new_comp":
                self.new_comp(event)
                self.mode_now = "view"
            elif self.mode_now == "view":
                self.click_comp(event)
            elif self.mode_now == 'add_child':
                self.click_comp(event)
                self.mode_now = "view"
                
        self.on_mouse_click = False
        self.on_mouse_drag = False
        self.ondrag_comp = False
    def new_comp(self,event):
        x, y = float(event.xdata), float(event.ydata)
        while True:
            new_ID = str(uuid.uuid4())
            if len(self.selected_map_data['components']) == 0:
                break
            if new_ID not in self.selected_map_data['components'].keys():
                break
            
        self.mapper_dict[self.selected_map]['components'][new_ID] = {'x':x-10, 'y':y-5,
                                                                     'width':20, 'height':10, 
                                                                     'color':'white', 'alpha':0.5,
                                                                     'label':'---', 'fontsize':20}
        del(self.mapper_dict[self.selected_map]['components']['temp'])
        self.selected_map_data_df = self.selected_map_data_df.drop('temp', errors = 'ignore')
        
        self.view_customized_map()
    
    def click_comp(self,event):
        x, y = int(event.xdata), int(event.ydata)
        sub_df = self.selected_map_data_df[(x>=self.selected_map_data_df['x'])*(y>=self.selected_map_data_df['y'])]
        sub_df = sub_df[(x-sub_df['x']<sub_df['width'])*(y-sub_df['y']<sub_df['height'])]
        if len(sub_df) >1 :
            idx = np.argmin((x-sub_df['x'].values-sub_df['width']/2)**2+(y-sub_df['y'].values-sub_df['height'])**2)
            sub_df = sub_df.iloc[idx:idx+1]
        if self.mode_now != "add_child":
            if len(sub_df) == 1:
                self.onclick_comp = sub_df.index[0]
            else:
                self.onclick_comp = False
            self.view_customized_map()
            self.load_detail()
        else:
            print(sub_df)
            if len(sub_df) == 1:
                child_label = self.mapper_dict[self.selected_map]['components'][sub_df.index[0]]['label']
                self.mapper_dict[self.selected_map]['relatives'].loc[len(self.mapper_dict[self.selected_map]['relatives'])] = [self.onclick_comp, sub_df.index[0],child_label]
                self.customize_detail.listbox_child.insert(tk.END, f"{child_label}: {sub_df.index[0]}")

In [8]:
class Kegg_mapper(Table_loader, KEGG_caller, Customized_mapper):
    def __init__(self):
        Table_loader.__init__(self)
        KEGG_caller.__init__(self)
        Customized_mapper.__init__(self)
        
        pass
        
    def window_controller(self):
        self.controller = tk.Tk()
        self.controller.title('Controller')
        self.controller.geometry("300x200")
        
        
        self.controller_btn_1 = tk.Button(self.controller, text = 'Load Table', command = self.load_table)
        self.controller_btn_1.pack(pady=20)

        self.controller_btn_2 = tk.Button(self.controller, text = 'Get Pathway', command = self.get_pathway)
        self.controller_btn_2.pack(pady=20)
        
        self.controller_btn_3 = tk.Button(self.controller, text = 'Customed Maps', command = self.load_customizer)
        self.controller_btn_3.pack(pady=20)
        
        self.controller.mainloop()
        

    
            
    

In [23]:
A.window_controller()


362 496
Index(['41AA0C31'], dtype='object')
[] []
[] []
[] []
[] []
[] []
[] []
643 327
Index(['D8B3C45F'], dtype='object')
[487, 488, 489] ['ATP2A1', 'ATP2A2', 'ATP2A3']
['ATP2A1', 'ATP2A2', 'ATP2A3']
All_by_disease|FC C1
103 755 179 1129
178 700 262 1022
145 772 158 1070
98 815 37 1132
193 271
Index(['0889292D'], dtype='object')
[80228, 84876, 93129] ['ORAI2', 'ORAI1', 'ORAI3']
['ORAI2', 'ORAI1', 'ORAI3']
662 432
Index([], dtype='object')
[3706, 3707, 80271, 107, 109, 114, 5136, 5137, 5153, 4842, 4843, 4846, 7416, 7417, 7419, 10105, 6543, 6546, 57118, 57172, 814, 815, 816, 817, 818, 8536, 5530, 5532, 5533, 5534, 5535, 5578, 5579, 5582, 2185, 219931, 53373, 952, 51196, 1956, 2064, 2065, 2066, 2260, 2261, 2263, 2264, 2321, 2324, 3791, 4233, 4486, 4914, 4915, 4916, 5156, 5159, 5979, 2767, 2769, 2776, 9630, 116443, 116444, 22953, 2902, 2903, 2904, 2905, 2906, 5023, 5024, 5025, 5026, 5027, 9127, 5335, 5336, 56848, 8877, 89869, 4638, 85366, 91807, 113026, 5333, 84812, 3708, 3709, 3710, 525

Traceback (most recent call last):
  File "c:\Users\test\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 287, in process
    func(*args, **kwargs)
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 173, in on_canvas_click
    x, y = int(event.xdata), int(event.ydata)
TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'


85 916 115 903
27 1024 65 986
96 894 121 857
40 998 76 959
0 1039 22 986
127 958 50 821
34 1031 29 954
0 1039 3 986
0 1039 0 986
0 1039 0 986
147 336
Index(['5BA3896B'], dtype='object')
[] []
[] []
[] []
[] []
144 325
Index([], dtype='object')
[10054, 10055, 6921, 6923, 9978, 23291, 26232, 26272, 55294, 6502, 8945, 23221, 55958, 90293, 9817, 9886, 1161, 1643, 55070, 64326, 8651, 9021, 7428, 26091, 55284, 140739, 9039, 246184, 10393, 8697, 8881, 51434, 51433, 29945, 996, 51343, 991, 29882, 51529, 26259, 8452, 8453, 6500, 8454, 9820, 8065, 9616, 1642, 8450, 8451, 4591, 22954, 4281, 55120, 672, 23295, 326, 10401, 51588, 8554, 9063, 329, 330, 331, 79444, 25898, 4214, 7189, 5371, 6477, 5071, 23624, 867, 868, 4193, 22888, 27339, 7337, 148581, 51619, 7321, 7322, 7323, 389898, 7334, 11059, 57154, 64750, 9690, 51465, 89910, 27338, 11065, 7319, 83737, 3093, 118424, 9246, 7332, 7327, 7326, 55236, 7317, 23759, 8924, 54926, 997, 10273, 8925, 11060, 10277, 51366, 9320, 9354, 10075, 65264, 57448, 233

Traceback (most recent call last):
  File "c:\Users\test\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 287, in process
    func(*args, **kwargs)
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 173, in on_canvas_click
    x, y = int(event.xdata), int(event.ydata)
TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'


834 454
Index([], dtype='object')
[10054, 10055, 6921, 6923, 9978, 23291, 26232, 26272, 55294, 6502, 8945, 23221, 55958, 90293, 9817, 9886, 1161, 1643, 55070, 64326, 8651, 9021, 7428, 26091, 55284, 140739, 9039, 246184, 10393, 8697, 8881, 51434, 51433, 29945, 996, 51343, 991, 29882, 51529, 26259, 8452, 8453, 6500, 8454, 9820, 8065, 9616, 1642, 8450, 8451, 4591, 22954, 4281, 55120, 672, 23295, 326, 10401, 51588, 8554, 9063, 329, 330, 331, 79444, 25898, 4214, 7189, 5371, 6477, 5071, 23624, 867, 868, 4193, 22888, 27339, 7337, 148581, 51619, 7321, 7322, 7323, 389898, 7334, 11059, 57154, 64750, 9690, 51465, 89910, 27338, 11065, 7319, 83737, 3093, 118424, 9246, 7332, 7327, 7326, 55236, 7317, 23759, 8924, 54926, 997, 10273, 8925, 11060, 10277, 51366, 9320, 9354, 10075, 65264, 57448, 23327, 4734, 134111, 55585, 92912, 63893, 9040, 7329, 7328, 7320, 25847, 10477, 7324, 7325, 6233, 7311, 7314, 7316, 25906, 119504] ['UBA2', 'SAE1', 'ELOC', 'ELOB', 'RBX1', 'FBXW11', 'FBXO2', 'FBXO4', 'FBXW7', 'SKP

...

Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 202, in on_canvas_click
    self.listbox_gene.selection_clear(0,tk.END)
  File "c:\Users\test\anaconda3\lib\tkinter\__init__.py", line 3226, in selection_clear
    self.tk.call(self._w,
_tkinter.TclError: invalid command name ".!frame2.!listbox"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 202, in on_canvas_click
    self.listbox_gene.selection_clear(0,tk.END)
  File "c:\Users\test\anaconda3\lib\tkinter\__init__.py", line 3226, in selection_clear
    self.tk.call(self._w,
_tkinter.TclError: invalid command name ".!frame2.!listbox"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 202, in on_canvas_click
    s

172 961 90 643
23 970 29 693
0 982 0 720
0 994 0 720
181 976 58 634
326 962 104 565
214 977 92 646
79 995 75 720
0 1017 49 720
0 1029 20 720
214 865
Index(['E29A1C76'], dtype='object')
[1737] ['DLAT']
['DLAT']
188 1012 72 632
51 1029 4 676
0 1029 0 720
686 237
Index(['8B9160B1'], dtype='object')
[] []
[] []
[] []
[] []
682 236
Index(['8B9160B1'], dtype='object')
[5160, 5161, 5162] ['PDHA1', 'PDHA2', 'PDHB']
['PDHA1', 'PDHA2', 'PDHB']
574 241
Index(['928EDBA7'], dtype='object')
[5160, 5161, 5162] ['PDHA1', 'PDHA2', 'PDHB']
['PDHA1', 'PDHA2', 'PDHB']
572 233
Index(['928EDBA7'], dtype='object')
[5160, 5161, 5162] ['PDHA1', 'PDHA2', 'PDHB']
['PDHA1', 'PDHA2', 'PDHB']
466 243
Index(['9BB566CA'], dtype='object')
[1737] ['DLAT']
['DLAT']
541 278
Index(['7AA7EE61'], dtype='object')
[1738] ['DLD']
['DLD']
459 243
Index(['9BB566CA'], dtype='object')
[1737] ['DLAT']
['DLAT']
529 278
Index(['7AA7EE61'], dtype='object')
[1738] ['DLD']
['DLD']
588 237
Index(['928EDBA7'], dtype='object')
[5160, 5161,

Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 202, in on_canvas_click
    self.listbox_gene.selection_clear(0,tk.END)
  File "c:\Users\test\anaconda3\lib\tkinter\__init__.py", line 3226, in selection_clear
    self.tk.call(self._w,
_tkinter.TclError: invalid command name ".!frame2.!listbox"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 202, in on_canvas_click
    self.listbox_gene.selection_clear(0,tk.END)
  File "c:\Users\test\anaconda3\lib\tkinter\__init__.py", line 3226, in selection_clear
    self.tk.call(self._w,
_tkinter.TclError: invalid command name ".!frame2.!listbox"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 202, in on_canvas_click
    s

427 342
Index(['E0A0FC68'], dtype='object')
[1431] ['CS']
['CS']
561 343
Index(['69BD69BD'], dtype='object')
[48, 50] ['ACO1', 'ACO2']
['ACO1', 'ACO2']
686 345
Index(['F1BFD5B8'], dtype='object')
[48, 50] ['ACO1', 'ACO2']
['ACO1', 'ACO2']
673 347
Index(['F1BFD5B8'], dtype='object')
[48, 50] ['ACO1', 'ACO2']
['ACO1', 'ACO2']
722 400
Index(['7AA2D55E'], dtype='object')
[3417, 3418] ['IDH1', 'IDH2']
['IDH1', 'IDH2']
730 512
Index(['FEA3CFF4'], dtype='object')
[3417, 3418] ['IDH1', 'IDH2']
['IDH1', 'IDH2']
651 579
Index(['57BED2C3'], dtype='object')
[4967, 55753] ['OGDH', 'OGDHL']
['OGDH', 'OGDHL']
468 611
Index([], dtype='object')
[1738, 4967, 55753, 1743, 8801, 8802, 8803, 3417, 3418, 3419, 3420, 3421, 47, 2271, 48, 50, 1431, 4190, 4191, 5091, 5160, 5161, 5162, 1737, 5105, 5106, 6389, 6390, 6391, 6392] ['DLD', 'OGDH', 'OGDHL', 'DLST', 'SUCLG2', 'SUCLG1', 'SUCLA2', 'IDH1', 'IDH2', 'IDH3A', 'IDH3B', 'IDH3G', 'ACLY', 'FH', 'ACO1', 'ACO2', 'CS', 'MDH1', 'MDH2', 'PC', 'PDHA1', 'PDHA2', 'PDHB'

Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 202, in on_canvas_click
    self.listbox_gene.selection_clear(0,tk.END)
  File "c:\Users\test\anaconda3\lib\tkinter\__init__.py", line 3226, in selection_clear
    self.tk.call(self._w,
_tkinter.TclError: invalid command name ".!frame2.!listbox"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 202, in on_canvas_click
    self.listbox_gene.selection_clear(0,tk.END)
  File "c:\Users\test\anaconda3\lib\tkinter\__init__.py", line 3226, in selection_clear
    self.tk.call(self._w,
_tkinter.TclError: invalid command name ".!frame2.!listbox"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 202, in on_canvas_click
    s

In [9]:
A = Kegg_mapper()
A.window_controller()

{0}
18 841 118 694
34 693 212 672
86 613 268 636
39 671 217 659
487 270
Index(['488F890D'], dtype='object')
[] []
[] []
[] []
[] []
[127] ['ADH4']
482 266
Index(['488F890D'], dtype='object')
[2821] ['GPI']
['GPI']
481 271
Index(['488F890D'], dtype='object')
[2821] ['GPI']
['GPI']
0 730 147 677
0 798 64 700
99 738 166 675
57 823 120 720
5 924 71 720
0 1029 23 720
69 893 51 609
124 784 73 520
168 696 91 448
204 626 99 385
176 682 92 436
142 749 84 497
101 830 74 570
52 927 61 656
0 1029 40 720
0 1029 16 720
0 1029 0 720
0 1029 0 720
47 870 94 670
84 743 170 630
113 640 231 599
135 557 281 575
118 625 242 595
97 706 196 620
71 802 142 651
38 915 88 698
0 1029 39 720
62 885 57 602
111 769 71 507
150 676 83 431
181 602 92 370
132 637 50 383
73 679 0 400
2 730 0 421
0 791 0 443
0 857 0 468
0 922 0 496
0 989 0 528
0 1029 0 566
0 1029 0 609
0 1029 0 656
143 966 103 628
258 916 186 606
352 878 255 591
427 848 311 580
364 869 267 590
288 894 215 603
196 924 153 619
90 964 91 650
0 1013 25 696
0 

Traceback (most recent call last):
  File "c:\Users\test\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 287, in process
    func(*args, **kwargs)
  File "C:\Users\test\AppData\Local\Temp\ipykernel_25148\2066429809.py", line 297, in zoom_in_out
    x, y = int(event.xdata), int(event.ydata)
TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'


137 702 0 740
44 722 0 833
0 737 0 939
82 672 107 858
145 617 201 801
435 338
Index(['E0A0FC68'], dtype='object')
[1431] ['CS']
['CS']
63 629 86 806
0 645 0 821
0 663 0 842
0 683 0 869
0 707 0 924
0 732 0 970
0 737 0 970
0 737 0 970
132 722 26 802
237 709 47 668
321 699 64 561
264 718 0 582
196 737 0 605
115 737 0 623
219 717 36 535
123 1147 87 1256
224 1043 156 1092
140 1123 111 1234
38 1218 57 1405
0 1281 0 1461


Traceback (most recent call last):
  File "c:\Users\test\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 287, in process
    func(*args, **kwargs)
  File "C:\Users\test\AppData\Local\Temp\ipykernel_25148\2066429809.py", line 173, in on_canvas_click
    x, y = int(event.xdata), int(event.ydata)
TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'


142 1167 17 1186
256 1076 31 966
347 1003 42 790
276 1064 22 920
142 1088 0 1042
0 1116 0 1187
191 1084 186 1136
37 1109 26 1166
0 1138 0 1203
0 1168 0 1242
0 1199 0 1287
0 1230 0 1375
0 1268 0 1461
146 1160 202 1370
254 1066 394 1329
138 1112 316 1438
0 1165 224 1461
0 1226 130 1461
0 1281 36 1461
188 1213 244 1384
48 1278 99 1461
0 1281 0 1461
0 1281 0 1461
204 1229 112 1281
41 1271 22 1424
0 1281 0 1461
105 1130 42 1211
202 1022 74 1009
283 939 99 847
213 1000 84 982
128 1073 63 1140
26 1160 0 1270
145 1053 21 1037
121 668
Index(['75D89CCA'], dtype='object')
97 588
Index(['6EBB4746'], dtype='object')
141 591
Index(['E6A9B5C0'], dtype='object')
[10398, 29895, 4632, 4633, 4634, 4635, 4636, 58498] ['MYL9', 'MYL11', 'MYL1', 'MYL2', 'MYL3', 'MYL4', 'MYL5', 'MYL7']
['MYL9', 'MYL11', 'MYL1', 'MYL2', 'MYL3', 'MYL4', 'MYL5', 'MYL7']
[10398, 29895, 4632, 4633, 4634, 4635, 4636, 58498] ['MYL9', 'MYL11', 'MYL1', 'MYL2', 'MYL3', 'MYL4', 'MYL5', 'MYL7']
[10398, 29895, 4632, 4633, 4634, 4635, 4636

...

Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_25148\2066429809.py", line 202, in on_canvas_click
    self.listbox_gene.selection_clear(0,tk.END)
  File "c:\Users\test\anaconda3\lib\tkinter\__init__.py", line 3226, in selection_clear
    self.tk.call(self._w,
_tkinter.TclError: invalid command name ".!frame2.!listbox"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_25148\2066429809.py", line 202, in on_canvas_click
    self.listbox_gene.selection_clear(0,tk.END)
  File "c:\Users\test\anaconda3\lib\tkinter\__init__.py", line 3226, in selection_clear
    self.tk.call(self._w,
_tkinter.TclError: invalid command name ".!frame2.!listbox"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\test\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 287, in process
    f

133 733 192 960
234 714 330 944
316 700 445 937
283 743 409 999
243 750 367 1075
199 750 315 1165
288 728 454 1134
260 750 358 1174
314 706 445 1098
98 748 146 1223
173 693 258 1119
920 376
Index([], dtype='object')
[5923, 5924, 11221, 1843, 1844, 1846, 1847, 1848, 1849, 1852, 80824, 1845, 5778, 5801, 84867, 5495, 356, 9448, 255189, 5321, 8605, 3925, 4137, 4790, 4791, 5970, 5971, 1147, 3551, 8517, 9020, 5594, 5595, 8649, 5605, 5604, 5894, 673, 5921, 4763, 5578, 5579, 5582, 5906, 5908, 5566, 5567, 5568, 9693, 5922, 22800, 22808, 3265, 3845, 4893, 6237, 10125, 10235, 115727, 25780, 6654, 6655, 2768, 55970, 2885, 1436, 1956, 1969, 2064, 2065, 2066, 2260, 2261, 2263, 2264, 2321, 2322, 2324, 3480, 3643, 3791, 3815, 4233, 4804, 4914, 4915, 5156, 5159, 5979, 7010, 10368, 10369, 27091, 27092, 55799, 59283, 59284, 59285, 774, 776, 777, 778, 779, 781, 782, 783, 784, 785, 786, 8911, 8912, 8913, 9254, 93589, 3164, 5598, 5607, 51701, 994, 8986, 9252, 7867, 9261, 468, 4208, 4149, 1649, 2005, 7157, 2

Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_25148\2066429809.py", line 202, in on_canvas_click
    self.listbox_gene.selection_clear(0,tk.END)
  File "c:\Users\test\anaconda3\lib\tkinter\__init__.py", line 3226, in selection_clear
    self.tk.call(self._w,
_tkinter.TclError: invalid command name ".!frame2.!listbox"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_25148\2066429809.py", line 202, in on_canvas_click
    self.listbox_gene.selection_clear(0,tk.END)
  File "c:\Users\test\anaconda3\lib\tkinter\__init__.py", line 3226, in selection_clear
    self.tk.call(self._w,
_tkinter.TclError: invalid command name ".!frame2.!listbox"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\test\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 287, in process
    f

332 754 404 1092
284 791 301 1127
226 835 177 1169
156 887 29 1219
72 949 0 1279
0 1025 0 1344
0 1046 0 1346
0 1046 0 1346
189 1026 206 1282
340 1010 368 1228
436 972 484 1172
360 1003 374 1199
268 1039 241 1231
159 1046 85 1273
34 1046 0 1328
0 1046 0 1346
146 982 90 1167
262 931 163 1024
174 976 16 1049
61 1023 0 1094
0 1046 0 1140
141 978 211 1123
254 923 382 1111
344 879 520 1103
268 910 432 1131
176 947 328 1167
68 993 211 1218
0 1046 85 1294
0 1046 0 1346
134 971 39 1116
41 1045 0 1198
0 1046 0 1287
91 928 222 1252
164 833 402 1226
86 889 315 1304
0 956 210 1346
159 924 264 1173
286 898 307 1034
383 873 361 942
328 916 257 954
261 967 136 972
178 1026 0 1003
78 1046 0 1043
0 1046 0 1089
121 958 216 1087
219 889 386 1083
768 706
Index(['DABAC81A'], dtype='object')
[1432, 5600, 5603, 6300] ['MAPK14', 'MAPK11', 'MAPK13', 'MAPK12']
['MAPK14', 'MAPK11', 'MAPK13', 'MAPK12']
915 597
Index(['60A2CCD7'], dtype='object')
[2002] ['ELK1']
['ELK1']
759 705
Index(['DABAC81A'], dtype='object')


Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_25148\2066429809.py", line 202, in on_canvas_click
    self.listbox_gene.selection_clear(0,tk.END)
  File "c:\Users\test\anaconda3\lib\tkinter\__init__.py", line 3226, in selection_clear
    self.tk.call(self._w,
_tkinter.TclError: invalid command name ".!frame2.!listbox"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_25148\2066429809.py", line 202, in on_canvas_click
    self.listbox_gene.selection_clear(0,tk.END)
  File "c:\Users\test\anaconda3\lib\tkinter\__init__.py", line 3226, in selection_clear
    self.tk.call(self._w,
_tkinter.TclError: invalid command name ".!frame2.!listbox"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\test\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 287, in process
    f

In [36]:
A.df_use

,Symbol,Description,fcdepend&ovp005_disease,fcdepend&ovp005_normal,fcdepend&ovp005_filtered,Disease|FC C1,Disease|FC C2,Disease|FC C3,Disease|FC C4,Disease|OVP C1,...,All_by_disease|OVP C3,All_by_disease|OVP C4,All_by_disease|OVP C5,All_by_disease|OVP C6,All_by_disease|PT C1,All_by_disease|PT C2,All_by_disease|PT C3,All_by_disease|PT C4,All_by_disease|PT C5,All_by_disease|PT C6
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
9,NAT1,N-Acetyltransferase 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NAT2,N-acetyltransferase 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,SERPINA3,serpin family A member 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,AADAC,arylacetamide deacetylase,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,AAMP,angio associated migratory cell protein,NaN,NaN,NaN,0.002897,-0.145167,0.050520,0.037638,0.993304,...,0.290255,0.761049,0.202529,0.487629,0.221720,0.000401,0.000925,0.016800,0.000873,0.178677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221424,LRRC73,leucine rich repeat containing 73,NaN,NaN,NaN,0.037655,-0.021883,0.031487,-0.041369,0.904042,...,0.966464,0.821598,0.232600,0.936678,0.897502,0.000841,0.327829,0.015924,0.000121,0.474174
196541,METTL21C,"methyltransferase 21C, AARS1 lysine",NaN,NaN,NaN,0.000800,0.265574,-0.061495,-0.060228,0.998859,...,0.547520,0.586342,0.848370,0.343945,0.685783,0.011034,0.022189,0.012583,0.861445,0.060631
163782,KANK4,KN motif and ankyrin repeat domains 4,NaN,NaN,NaN,-0.120601,-0.081420,0.093369,0.099221,0.474159,...,0.839937,0.602889,0.714261,0.718540,0.000648,0.000124,0.556779,0.000326,0.153182,0.097487


In [24]:
A.window_controller()


75 912 44 1120
115 784 79 940
246 253
Index([], dtype='object')
143 678 112 801
226 312
Index(['F696ABF9'], dtype='object')
[1436, 1956, 1969, 2064, 2065, 2066, 2260, 2261, 2263, 2264, 2321, 2322, 2324, 3480, 3643, 3791, 3815, 4233, 4804, 4914, 4915, 5156, 5159, 5979, 7010] ['CSF1R', 'EGFR', 'EPHA2', 'ERBB2', 'ERBB3', 'ERBB4', 'FGFR1', 'FGFR3', 'FGFR2', 'FGFR4', 'FLT1', 'FLT3', 'FLT4', 'IGF1R', 'INSR', 'KDR', 'KIT', 'MET', 'NGFR', 'NTRK1', 'NTRK2', 'PDGFRA', 'PDGFRB', 'RET', 'TEK']
['CSF1R', 'EGFR', 'EPHA2', 'ERBB2', 'ERBB3', 'ERBB4', 'FGFR1', 'FGFR3', 'FGFR2', 'FGFR4', 'FLT1', 'FLT3', 'FLT4', 'IGF1R', 'INSR', 'KDR', 'KIT', 'MET', 'NGFR', 'NTRK1', 'NTRK2', 'PDGFRA', 'PDGFRB', 'RET', 'TEK']
113 755 86 913
66 837 50 1042
7 932 6 1196
93 833 31 983
145 737 76 837
94 804 31 944
104 926 45 991
92 720 53 915
172 675 92 781
238 640 123 674
189 672 84 746
140 720 32 827
108 785 0 920
99 963 38 1029
245 936 72 864
363 915 99 732
457 899 120 627
534 888 137 543
515 940 101 589
459 969 62 647
390

Traceback (most recent call last):
  File "c:\Users\test\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 287, in process
    func(*args, **kwargs)
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 173, in on_canvas_click
    x, y = int(event.xdata), int(event.ydata)
TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'


25 877 0 1081
0 951 0 1250
45 806 68 1068
123 731 83 883
78 807 12 972
150 733 45 813
67 766 0 869
0 808 0 932
0 857 0 997
109 795 152 950
15 838 88 1045
0 889 15 1163
0 960 0 1242
0 1036 0 1328
118 947 225 1287
104 591 90 893
23 608 17 980
0 609 0 1003
0 609 0 1003
910 35
Index([], dtype='object')
46 533 140 943
15 599 29 992
53 521 196 966
83 457 332 948
105 405 443 936
122 362 533 927
136 328 605 920
147 301 662 914
124 309 644 947
97 319 623 986
65 331 598 1003
26 345 571 1003
0 365 540 1003
0 388 508 1003
0 412 474 1003
0 437 437 1003
0 464 397 1003
0 493 354 1003
0 524 307 1003
0 557 257 1003
0 592 203 1003
0 609 145 1003
106 593 185 872
187 576 225 775
124 591 191 851
49 609 151 943
0 609 103 1003
125 1150 128 1296
229 1049 228 1162
130 1114 175 1295
12 1193 110 1454
151 1096 123 1198
261 1017 145 1005
349 954 162 850
294 1020 44 870
228 1099 0 894
149 1194 0 915
240 1076 21 753
312 981 37 623
104 594
Index(['6EBB4746'], dtype='object')
[] []
[] []
[] []
[] []
110 590
Index(['6E

Traceback (most recent call last):
  File "c:\Users\test\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 287, in process
    func(*args, **kwargs)
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 173, in on_canvas_click
    x, y = int(event.xdata), int(event.ydata)
TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'


1009 118
Index([], dtype='object')
72 988 115 1043
745 584
Index(['6FB83FE6'], dtype='object')
[] []
[] []
[] []
[] []
[] []
[] []
732 581
Index(['6FB83FE6'], dtype='object')
[10891] ['PPARGC1A']
['PPARGC1A']
0 1019 3 1116
0 1060 0 1160
0 1104 0 1160
206 1090 111 1039
373 1080 181 924
281 1129 95 986
170 1145 0 1061
43 1145 0 1151
0 1145 0 1160
0 1145 0 1160
24 940 160 1088
42 775 290 1033
57 643 399 994
67 536 495 971
3 566 450 1022
0 592 397 1083
0 620 334 1157
0 649 258 1160
164 922 235 1358
291 898 422 1320
392 877 571 1290
471 859 691 1266
534 844 787 1247
1100 686
Index(['015A98E3'], dtype='object')
[] []
[] []
[] []
[] []
1107 691
Index(['015A98E3'], dtype='object')
[3098, 3099, 3101, 80201] ['HK1', 'HK2', 'HK3', 'HKDC1']
['HK1', 'HK2', 'HK3', 'HKDC1']
1096 752
Index(['02D21E7E'], dtype='object')
[2023, 2026, 2027] ['ENO1', 'ENO2', 'ENO3']
['ENO1', 'ENO2', 'ENO3']
1171 689
Index(['03DFDC54'], dtype='object')
[5211, 5213, 5214] ['PFKL', 'PFKM', 'PFKP']
['PFKL', 'PFKM', 'PFKP']
11

...

916 145
Index(['0AAD3872'], dtype='object')
[] []
913 145
Index(['0AAD3872'], dtype='object')
[6517] ['SLC2A4']
['SLC2A4']
157 534
Index(['E6AEADB4'], dtype='object')
[6389] ['SDHA']
[6389, 6390] ['SDHA', 'SDHB']
[6389, 6390] ['SDHA', 'SDHB']
[6389, 6390] ['SDHA', 'SDHB']
[6389, 6390] ['SDHA', 'SDHB']
[6389, 6390] ['SDHA', 'SDHB']
[6389, 6390] ['SDHA', 'SDHB']
[6389, 6390] ['SDHA', 'SDHB']
[6389, 6390, 6391] ['SDHA', 'SDHB', 'SDHC']
[6389, 6390, 6391, 6392] ['SDHA', 'SDHB', 'SDHC', 'SDHD']
913 151
Index(['0AAD3872'], dtype='object')
[] []
921 148
Index(['0AAD3872'], dtype='object')
[6517] ['SLC2A4']
['SLC2A4']
63 886 68 644
114 772 122 583
18 807 102 655
0 849 79 720
0 945 6 720
0 1024 0 720
142 961 43 619
251 906 80 541
159 945 67 620
47 990 55 718
0 1029 42 720
53 877 123 665
94 753 190 624
124 652 245 593
147 569 291 569
84 591 282 616
8 617 272 672
0 648 254 720
0 677 223 720
0 707 188 720
0 740 147 720
325 186
Index(['269728EB'], dtype='object')
[] []
[] []
[] []
[] []
339 192
Ind

Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 202, in on_canvas_click
    self.listbox_gene.selection_clear(0,tk.END)
  File "c:\Users\test\anaconda3\lib\tkinter\__init__.py", line 3226, in selection_clear
    self.tk.call(self._w,
_tkinter.TclError: invalid command name ".!frame2.!listbox"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\test\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 287, in process
    func(*args, **kwargs)
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 297, in zoom_in_out
    x, y = int(event.xdata), int(event.ydata)
TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'
Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 202, in on_canvas_click
    self.listbox_gene.selection_cle

51 806 141 606
91 695 143 515
123 606 144 442
149 535 146 384
127 590 141 427
101 656 135 478
70 736 128 540
32 832 120 614
0 947 110 703
0 1029 98 720
0 1029 87 720
49 872 157 664
8 995 99 707
0 1029 30 720
51 874 110 662
91 750 175 617
220 216
Index(['279726FE'], dtype='object')
[3098, 3099, 3101, 80201] ['HK1', 'HK2', 'HK3', 'HKDC1']
['HK1', 'HK2', 'HK3', 'HKDC1']
315 191
Index(['269728EB'], dtype='object')
[2538, 57818, 92579] ['G6PC1', 'G6PC2', 'G6PC3']
['G6PC1', 'G6PC2', 'G6PC3']
585 358
Index([], dtype='object')
[226, 229, 230, 217, 219, 223, 224, 501, 218, 221, 222, 10327, 124, 125, 126, 127, 128, 130, 131, 5160, 5161, 5162, 1737, 3939, 3945, 3948, 92483, 5313, 5315, 2023, 2026, 2027, 5223, 5224, 669, 2597, 26330, 7167, 5211, 5213, 5214, 2203, 8789, 2821, 5236, 55276, 3098, 3099, 3101, 80201, 2645, 130589, 2538, 57818, 92579, 1738, 5230, 5232, 5105, 5106, 55902, 84532, 83440, 9562] ['ALDOA', 'ALDOB', 'ALDOC', 'ALDH2', 'ALDH1B1', 'ALDH9A1', 'ALDH3A2', 'ALDH7A1', 'ALDH3A1', 'ALDH

Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 202, in on_canvas_click
    self.listbox_gene.selection_clear(0,tk.END)
  File "c:\Users\test\anaconda3\lib\tkinter\__init__.py", line 3226, in selection_clear
    self.tk.call(self._w,
_tkinter.TclError: invalid command name ".!frame2.!listbox"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\test\anaconda3\lib\site-packages\matplotlib\cbook\__init__.py", line 287, in process
    func(*args, **kwargs)
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 297, in zoom_in_out
    x, y = int(event.xdata), int(event.ydata)
TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'
Traceback (most recent call last):
  File "C:\Users\test\AppData\Local\Temp\ipykernel_77592\2066429809.py", line 202, in on_canvas_click
    self.listbox_gene.selection_cle

122 650 186 540
141 563 211 495
121 628 189 530
97 705 161 570
66 795 126 617
28 902 83 672
0 1023 36 720
69 888 98 645
19 1001 47 703
0 1029 0 720
73 896 75 651
134 793 134 595
183 711 181 550
161 794 130 573
122 882 82 614
66 978 36 675
87 817 139 650
104 688 221 630
117 585 287 614
128 503 340 601
137 437 382 591
144 384 416 583
150 342 443 576
125 356 437 597
93 371 430 622
53 387 420 650
5 406 408 684
0 439 390 720
0 475 369 720
0 515 346 720
0 561 314 720
0 616 270 720
0 677 221 720
0 744 167 720
0 818 107 720
0 910 59 720
52 780 70 599
93 675 79 502
62 760 35 542
21 859 0 590
59 729 84 556
89 625 153 531
113 542 213 515
145 488 250 492
171 445 281 475
150 479 252 485
125 520 217 496
95 569 175 510
59 628 125 527
16 699 66 548
0 784 0 575
53 680 44 504
95 597 80 448
329 193
Index(['269728EB'], dtype='object')
[2538, 57818, 92579] ['G6PC1', 'G6PC2', 'G6PC3']
['G6PC1', 'G6PC2', 'G6PC3']
149 550 96 390
192 512 109 344
161 545 91 373
125 585 65 404
81 633 34 441
11 673 0 470
0 721 0 

In [142]:
A.window_controller()


In [ ]:
fc_df = pd.read_excel('241203_fc_ovp_pt_all_fc_028_ovp005.xlsx',index_col=0, header = [0,1,2])

In [19]:
fc_df = pd.read_csv('241217_fc_ovp_pt_all_fc_depend_pt_001.csv',index_col=0)
fc_df

C:\Users\test\AppData\Local\Temp\ipykernel_96576\3259452084.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  fc_df = pd.read_csv('241217_fc_ovp_pt_all_fc_depend_pt_001.csv',index_col=0)


,Symbol,Description,fcdepend&pt001_disease,fcdepend&pt001_normal,fcdepend&pt001_filtered,Disease|FC C1,Disease|FC C2,Disease|FC C3,Disease|FC C4,Disease|OVP C1,...,All_by_disease|OVP C3,All_by_disease|OVP C4,All_by_disease|OVP C5,All_by_disease|OVP C6,All_by_disease|PT C1,All_by_disease|PT C2,All_by_disease|PT C3,All_by_disease|PT C4,All_by_disease|PT C5,All_by_disease|PT C6
9,NAT1,N-Acetyltransferase 1,NaN,NaN,NaN,0.098503,-0.052931,0.052318,-0.104326,0.188021,...,0.527462,0.079511,0.822475,0.999669,0.000002,0.004252,0.004585,3.433864e-11,0.123520,0.971738
10,NAT2,N-acetyltransferase 2,NaN,NaN,NaN,-0.029256,-0.017075,0.027334,0.021476,0.866056,...,0.972799,0.995920,0.731176,0.116738,0.000020,0.010880,0.687458,4.634450e-01,0.168840,0.000003
12,SERPINA3,serpin family A member 3,NaN,NaN,NaN,-0.005193,0.021748,-0.046765,0.009288,0.936427,...,0.678751,0.899565,0.891215,0.772490,0.000565,0.231997,0.095922,1.696642e-03,0.155630,0.438754
13,AADAC,arylacetamide deacetylase,NaN,NaN,NaN,-0.061312,0.038930,-0.035546,0.049767,0.792108,...,0.622967,0.923175,0.840432,0.423954,0.067088,0.007007,0.040831,6.634070e-01,0.181548,0.005778
14,AAMP,angio associated migratory cell protein,NaN,NaN,NaN,0.041863,0.075770,-0.151716,0.001524,0.926326,...,0.347453,0.855227,0.828198,0.215502,0.232504,0.002175,0.002942,1.672004e-01,0.800455,0.008439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221424,NaN,NaN,NaN,NaN,NaN,-0.022372,0.026594,-0.033830,0.029864,0.948512,...,0.490451,0.981815,0.654476,0.919080,0.061693,0.097230,0.001848,9.225727e-01,0.030015,0.383881
196541,NaN,NaN,NaN,NaN,NaN,-0.068907,-0.035029,0.248376,0.004673,0.765574,...,0.270453,0.778696,0.101790,0.581641,0.000513,0.000291,0.031646,4.346776e-01,0.001450,0.260835
163782,NaN,NaN,NaN,NaN,NaN,0.117731,0.105591,-0.100999,-0.138569,0.418679,...,0.376750,0.317616,0.715346,0.911562,0.003256,0.111118,0.011598,1.686215e-03,0.153191,0.373659
163786,NaN,NaN,NaN,NaN,NaN,0.074079,-0.144903,0.086385,-0.025259,0.835280,...,0.769509,0.986954,0.931068,0.888236,0.574711,0.315281,0.640859,9.441955e-01,0.390323,0.228187


In [26]:
symbol_df = fc_df.iloc[:,:2]
signature_info_df = fc_df.iloc[:,2:5]
fc_df_disease_only = fc_df.iloc[:,5:5+4*3]
fc_df_normal_only = fc_df.iloc[:,5+4*3:5+4*3+2*3]
fc_df_all = fc_df.iloc[:,5+4*3+2*3:5+4*3+2*3+6*3]
fc_df_all

,All_by_disease|FC C1,All_by_disease|FC C2,All_by_disease|FC C3,All_by_disease|FC C4,All_by_disease|FC C5,All_by_disease|FC C6,All_by_disease|OVP C1,All_by_disease|OVP C2,All_by_disease|OVP C3,All_by_disease|OVP C4,All_by_disease|OVP C5,All_by_disease|OVP C6,All_by_disease|PT C1,All_by_disease|PT C2,All_by_disease|PT C3,All_by_disease|PT C4,All_by_disease|PT C5,All_by_disease|PT C6
9,0.063708,-0.054273,0.041717,-0.086566,0.014330,-0.000138,0.420862,0.535715,0.527462,0.079511,0.822475,0.999669,0.000002,0.004252,0.004585,3.433864e-11,0.123520,0.971738
10,-0.043636,-0.035805,0.006199,0.000425,0.029338,0.071306,0.583082,0.671797,0.972799,0.995920,0.731176,0.116738,0.000020,0.010880,0.687458,4.634450e-01,0.168840,0.000003
12,-0.002848,0.004668,-0.044260,-0.003476,-0.007962,0.041334,0.953377,0.965554,0.678751,0.899565,0.891215,0.772490,0.000565,0.231997,0.095922,1.696642e-03,0.155630,0.438754
13,-0.064811,0.025363,-0.045157,0.024271,0.014787,0.047805,0.781504,0.717769,0.622967,0.923175,0.840432,0.423954,0.067088,0.007007,0.040831,6.634070e-01,0.181548,0.005778
14,0.048589,0.072334,-0.082558,0.022649,-0.049136,-0.116483,0.883413,0.430441,0.347453,0.855227,0.828198,0.215502,0.232504,0.002175,0.002942,1.672004e-01,0.800455,0.008439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221424,-0.032575,0.027732,-0.042617,0.011740,0.024449,0.012173,0.884360,0.811239,0.490451,0.981815,0.654476,0.919080,0.061693,0.097230,0.001848,9.225727e-01,0.030015,0.383881
196541,-0.114771,-0.113271,0.181529,-0.054659,0.185602,0.070341,0.433020,0.255040,0.270453,0.778696,0.101790,0.581641,0.000513,0.000291,0.031646,4.346776e-01,0.001450,0.260835
163782,0.069284,0.064145,-0.092431,-0.103038,0.030636,0.013183,0.631385,0.656218,0.376750,0.317616,0.715346,0.911562,0.003256,0.111118,0.011598,1.686215e-03,0.153191,0.373659
163786,0.055113,-0.120273,0.066062,-0.009810,-0.008068,-0.012934,0.919533,0.581635,0.769509,0.986954,0.931068,0.888236,0.574711,0.315281,0.640859,9.441955e-01,0.390323,0.228187


In [27]:
image_array = Mapper.image_array
gene_dict = Mapper.gene_dict
rectangles = Mapper.rectangles
rect_df = Mapper.rect_df

In [12]:
cmap_custom=LinearSegmentedColormap.from_list('rg',["limegreen", "w", "r"], N=128) 
x_min, y_min = 0, 0
x_max, y_max = image_array.shape[0], image_array.shape[1]
x_min_now, y_min_now = 0, 0
x_max_now, y_max_now = image_array.shape[0], image_array.shape[1]

root = tk.Tk()
left_frame = tk.Frame(root, width=600, height=400, bg="white")
right_frame = tk.Frame(root, width=400, height=400, bg="lightgray")
left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=False)
right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=False)

fig = Figure(figsize=(image_array.shape[0]/100, image_array.shape[1]/100), dpi=100)
ax = fig.add_subplot(111)
ax.imshow(image_array,origin='lower')
ax.axis('off')
ax.set_ylim(x_max_now, x_min_now)
ax.set_xlim(y_min_now, y_max_now)
for rect in rectangles:
    x, y, width, height = rect
    # Rectangle 객체 생성 및 추가
    rect_patch = Rectangle(
        (x, y), width-x, height-y, 
        edgecolor='red', facecolor='none', linewidth=2
    )
    ax.add_patch(rect_patch)

canvas_main = FigureCanvasTkAgg(fig, master=left_frame)
canvas_main.get_tk_widget().pack()

fig_sub = Figure(figsize=(5, 5), dpi=100)
ax_sub = fig_sub.add_subplot(111)
canvas_sub = FigureCanvasTkAgg(fig_sub, master=right_frame)
canvas_sub.get_tk_widget().pack()
def on_sub_canvas_click(event):
    x, y = int(event.xdata), int(event.ydata)
    selected_cluster = (x+0.5)//1
    print(selected_cluster)
    ax.cla()
    ax.imshow(image_array,origin='lower')
    ax.axis('off')
    for rect, key_id in zip(rectangles, gene_dict.keys()):
        x, y, width, height = rect
        # Rectangle 객체 생성 및 추가
        rect_patch = Rectangle(
            (x, y), width-x, height-y, 
            edgecolor='red', facecolor='none', linewidth=2
        )
        ax.add_patch(rect_patch)
        target_entrez_ids = gene_dict[key_id]['hsa']
        run_rand = 0
        if len(target_entrez_ids) >1:
            run_rand = 1
        for i in target_entrez_ids:
            if int(i) in symbol_df.index:
                ax.scatter([rect[0]+run_rand*np.random.rand()*25],[rect[1]+run_rand*np.random.rand()*25],s = 50,
                        cmap = cmap_custom, c = fc_df_disease_only.loc[int(i)].iloc[int(selected_cluster)], vmin=-1.2, vmax=1.2, linewidths=1, edgecolors='black')
    ax.set_ylim(x_max_now, x_min_now)
    ax.set_xlim(y_min_now, y_max_now)
    canvas_main.draw()
    
def on_canvas_click(event):
    x, y = int(event.xdata), int(event.ydata)
    
    selected_id = rect_df[(rect_df['x1']<=x)*(rect_df['x2']>=x)*(rect_df['y1']<=y)*(rect_df['y2']>=y)].index[0]

    selected_df = fc_df_disease_only.iloc[:,:4].loc[[int(i) for i in gene_dict[selected_id]['hsa'] if int(i) in fc_df_disease_only.index]]
    selected_title = [f"{i}|{j}" for i,j in zip(gene_dict[selected_id]['hsa'], gene_dict[selected_id]['title']) if int(i) in fc_df_disease_only.index]
    ax_sub.cla()
    ax_sub.imshow(selected_df,
            interpolation='none', aspect = 'auto',
            cmap=cmap_custom, vmin=-1.2, vmax=1.2)
    ax_sub.set_yticks(range(len(selected_df)), selected_title)
    ax_sub.set_xticks(range(4), ['C1','C2','C3','C4'])
    ax_sub.set_xlim(-0.5, 3.5)
    ax_sub.set_ylim(-0.5, len(selected_df)-0.5)
    canvas_sub.draw()
def zoom_in_out(event):
    global x_min_now, x_max_now, y_min_now, y_max_now
    x, y = int(event.xdata), int(event.ydata)
    if event.button == 'up': # zoom in
        x_max_now = int(x+(x_max_now-x)*9/10)
        x_min_now = int(x-(x-x_min_now)*9/10)
        if x_min_now >= x_max_now:
            x_max_now = x_min_now + 1 
        if y != 0:
            y_max_now = int(y+(y_max_now-y)*9/10)
            y_min_now = int(y-(y-y_min_now)*9/10)
            if y_min_now >= y_max_now:
                y_max_now = y_min_now + 1 
    elif event.button == 'down': # zoom out
        x_max_now = int(x+(x_max_now-x)*11/10)
        x_min_now = int(x-(x-x_min_now)*11/10)
        x_max_now = min(x_max_now,x_max)
        x_min_now = max(x_min_now,x_min)
        if y != 0:
            y_max_now = int(y+(y_max_now-y)*11/10)
            y_min_now = int(y-(y-y_min_now)*11/10)
            y_max_now = min(y_max_now,y_max)
            y_min_now = max(y_min_now,y_min)
    ax.cla()
    ax.imshow(image_array,origin='lower')
    ax.axis('off')
    for rect in rectangles:
        x, y, width, height = rect
        # Rectangle 객체 생성 및 추가
        rect_patch = Rectangle(
            (x, y), width-x, height-y, 
            edgecolor='red', facecolor='none', linewidth=2
        )
        ax.add_patch(rect_patch)
    ax.set_ylim(x_max_now, x_min_now)
    ax.set_xlim(y_min_now, y_max_now)
    canvas_main.draw()
canvas_main.mpl_connect('button_press_event', on_canvas_click)
canvas_sub.mpl_connect('button_press_event', on_sub_canvas_click)
canvas_main.mpl_connect('scroll_event', zoom_in_out)


# tkinter 메인 루프 실행
root.mainloop()

NameError: name 'image_array' is not defined

In [ ]:
fc_df_disease_only

,Disease|FC C1,Disease|FC C2,Disease|FC C3,Disease|FC C4,Disease|OVP C1,Disease|OVP C2,Disease|OVP C3,Disease|OVP C4,Disease|PT C1,Disease|PT C2,Disease|PT C3,Disease|PT C4
9,0.098503,-0.052931,0.052318,-0.104326,0.188021,0.651171,0.528668,0.151959,8.580832e-10,0.023543,0.004712,2.815749e-10
10,-0.029256,-0.017075,0.027334,0.021476,0.866056,0.939419,0.732519,0.956168,2.207881e-02,0.438109,0.024843,3.774144e-01
12,-0.005193,0.021748,-0.046765,0.009288,0.936427,0.909090,0.665771,0.887095,1.291237e-03,0.314526,0.043204,7.964895e-04
13,-0.061312,0.038930,-0.035546,0.049767,0.792108,0.649548,0.750459,0.887539,6.112220e-02,0.005472,0.081438,3.459359e-01
14,0.041863,0.075770,-0.151716,0.001524,0.926326,0.531224,0.132381,0.996967,4.167173e-01,0.013832,0.000012,5.468827e-01
...,...,...,...,...,...,...,...,...,...,...,...,...
221424,-0.022372,0.026594,-0.033830,0.029864,0.948512,0.756906,0.678364,0.945631,2.257064e-01,0.013634,0.015843,4.639365e-01
196541,-0.068907,-0.035029,0.248376,0.004673,0.765574,0.747885,0.176027,0.995615,5.653427e-02,0.032371,0.003833,7.599283e-01
163782,0.117731,0.105591,-0.100999,-0.138569,0.418679,0.531300,0.343248,0.414259,3.202254e-04,0.069377,0.002247,1.764289e-03
163786,0.074079,-0.144903,0.086385,-0.025259,0.835280,0.519696,0.672809,0.972308,2.833685e-01,0.176744,0.332683,7.358576e-01


In [13]:
[int(i) for i in gene_dict[selected_id]['hsa']]

NameError: name 'gene_dict' is not defined

In [490]:
# tkinter 초기화
root = tk.Tk()
# Canvas 생성
canvas = tk.Canvas(root, width=600, height=400)
canvas.pack()